In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from stable_baselines3 import PPO
from enum import Enum
from sklearn.preprocessing import MinMaxScaler
import time
import hmac
import hashlib
import requests
import json
from finta import TA
import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.firestore_v1.base_query import FieldFilter

In [2]:
class Positions(int, Enum):
    SHORT = 0
    LONG = 1
    HOLD = 2
    TAKE = 3

In [63]:
def getTickerData(ticker, period, interval):
    data_query = 'symbol=BTCUSDT&interval=1d&limit=1200'
    HOST_URL = 'https://testnet.binancefuture.com'
    END_POINT = '/fapi/v1/markPriceKlines'
    url = HOST_URL + END_POINT + "?" + data_query
    data = requests.get(url=url).text
    data = json.loads(data)
    df_data = {'open': [], 'high': [], 'low': [], 'close': [], 'volume': []}
    for i in range(0, len(data)):
        open = data[i][1]
        high = data[i][2]
        low = data[i][3]
        close = data[i][4]
        df_data['open'].append(open)
        df_data['high'].append(high)
        df_data['low'].append(low)
        df_data['close'].append(close)
        
    df_data['volume'] = df_data['close']
        
    df = pd.DataFrame(df_data)
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)
    
    return df

In [64]:
def getTickerDataForLastPrice():
    data_query = 'symbol=BTCUSDT&interval=1h&limit=5'
    HOST_URL = 'https://testnet.binancefuture.com'
    END_POINT = '/fapi/v1/klines'
    url = HOST_URL + END_POINT + "?" + data_query
    data = requests.get(url=url).text
    data = json.loads(data)
    df_data = {'open': [], 'high': [], 'low': [], 'close': [], 'volume': []}
    for i in range(0, len(data)):
        open = data[i][1]
        high = data[i][2]
        low = data[i][3]
        close = data[i][4]
        df_data['open'].append(open)
        df_data['high'].append(high)
        df_data['low'].append(low)
        df_data['close'].append(close)
        
    df_data['volume'] = df_data['close']
        
    df = pd.DataFrame(df_data)
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)
    
    return df

In [4]:
def calculate_percentage_increase(final_value, starting_value):
    try:
        return 100 * ((final_value - starting_value) / starting_value)
    except:
        print(final_value, starting_value)

In [5]:
def supertrend_indicator(df, atr_period, multiplier):
    
    high = df['high']
    low = df['low']
    close = df['close']
    
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    hl2 = (high + low) / 2
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        else:
            supertrend[curr] = supertrend[prev]

            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return final_lowerband, final_upperband

In [6]:
def rsi_indicator(df):
    rsi = TA.RSI(df[['open', 'high', 'low', 'close']], 14)

    signals = []
    for i in range(0, len(rsi)):
        if rsi[i] > 60: # Default value: 70
            signals.append(Positions.SHORT)
        elif rsi[i] < 40: # Default value: 30
            signals.append(Positions.LONG)
        else:
            signals.append(Positions.HOLD)
        
    buy_signal = [True if signals[n]==1 else False for n in range(0, len(signals))]
    sell_signal = [True if signals[n]==-1 else False for n in range(0, len(signals))]
    
    return signals, buy_signal, sell_signal

In [7]:
def swing_detection(index, df):
    sh = []
    sl = []
    start = (index*2) - 1
    for i in range(index-1):
        sh.append(False)
        sl.append(False)
    for ci, row in df.iterrows():
        
        swing_high = False
        swing_low = False
        
        if ci < start:
            continue
        
        swing_point_high = df['high'][ci - index]
        swing_point_low = df['low'][ci - index]
        
        for i in range(0, start):
            swing_high = True
            if i < index:
                if df['high'][ci - i] > swing_point_high:
                    swing_high = False
                    break
            if i > index:
                if df['high'][ci - i] >= swing_point_high:
                    swing_high = False
                    break
            
        for i in range(0, start):
            swing_low = True
            if i < index:
                if df.low[ci - i] < swing_point_low: 
                    swing_low = False
                    break  
            if i > index:
                if df.low[ci - i] <= swing_point_low: 
                    swing_low = False
                    break 
            
        sh.append(swing_high)
        sl.append(swing_low)
        
    for i in range(index):
        sh.append(False)
        sl.append(False)
        
    current_sh = 0
    current_sl = 0
    sh_nums = []
    sl_nums = []
    for i, row in df.iterrows():
        if sh[i] == True:
            current_sh = df.high[i]
        if sl[i] == True:
            current_sl = df.low[i]
        sh_nums.append(current_sh)
        sl_nums.append(current_sl)
    return sh, sl, sh_nums, sl_nums

In [8]:
def money_flow_index_indicator(df, period=14):
    # Calculate typical price (TP) for each period
    df['TP'] = (df['high'] + df['low'] + df['close']) / 3

    # Calculate raw money flow (RMF) for each period
    df['RMF'] = df['TP'] * df['volume']

    # Calculate positive and negative money flow
    df['PMF'] = 0.0
    df['NMF'] = 0.0

    for i in range(1, len(df)):
        if df.at[i, 'TP'] > df.at[i - 1, 'TP']:
            df.at[i, 'PMF'] = df.at[i, 'TP'] * df.at[i, 'volume']
        elif df.at[i, 'TP'] < df.at[i - 1, 'TP']:
            df.at[i, 'NMF'] = df.at[i, 'TP'] * df.at[i, 'volume']

    # Calculate money flow ratio (MFR)
    df['MFR'] = df['PMF'].rolling(window=period).sum() / df['NMF'].rolling(window=period).sum()

    # Calculate Money Flow Index (MFI)
    df['MFI'] = 100 - (100 / (1 + df['MFR']))

    # Remove temporary columns
    df.drop(['TP', 'RMF', 'PMF', 'NMF', 'MFR'], axis=1, inplace=True)

    return df['MFI'].values

In [9]:
def produce_prediction(df, window):  
    prediction = (df.shift(window)['close'] <= df['close'])
    
    return prediction.astype(int)

In [10]:
def preprocess_data(df):
    scaler = MinMaxScaler()

    mfi_indicator = money_flow_index_indicator(df)
    close_b = produce_prediction(df, 1)
    rsi_signals, _, _ = rsi_indicator(df)
    sh, sl, sh_nums, sl_nums = swing_detection(5, df)
    final_lowerband, final_upperband = supertrend_indicator(df, 10, 3)
    fu_modified = [
        Positions.LONG if not np.isnan(lowerband) else Positions.SHORT
        for lowerband in final_lowerband
    ]

    df["close_binary"] = close_b
    df["mfi"] = mfi_indicator
    df["sh_nums"] = sh_nums
    df["sl_nums"] = sl_nums
    df["supertrend"] = fu_modified
    df["rsi_signals"] = rsi_signals

    df = df.drop(columns={"volume", "open", "high", "low", "MFI"})
    df = df.dropna()

    df[["mfi"]] = scaler.fit_transform(df[["mfi"]])

    return df

In [60]:
trades_done = []
model = PPO.load("saved_models/best_model_800000_LSTM4.zip")

c:\Users\damem\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object _last_lstm_states. Consider using `custom_objects` argument to replace this object.
Exception: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.
  warnings.warn(


In [65]:
df = pd.DataFrame()
while df.shape[0] == 0:
     df = getTickerData("btc-usd", "max", "1d")
df = df.rename(
    columns={
        "Close": "close",
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Volume": "volume",
        "Datetime": "date",
        "Date": "date",
    }
)
df = preprocess_data(df)
df = df[0 : df.shape[0] - 1]

In [66]:
dfflp = getTickerDataForLastPrice()['open'].iloc[-1]
print(dfflp)

46600.0


In [13]:
df.tail(50)

,close,close_binary,mfi,sh_nums,sl_nums,supertrend,rsi_signals
1149,43968.691395,1,0.482584,44351.349092,40629.650427,1,0
1150,43725.300000,0,0.482833,44351.349092,40629.650427,1,0
1151,43013.700000,0,0.484283,44351.349092,40629.650427,1,2
1152,43611.900000,1,0.566466,44351.349092,40629.650427,1,2
1153,42538.600000,0,0.564004,44351.349092,40629.650427,1,2
1154,43430.477292,1,0.565082,44351.349092,40629.650427,1,2
1155,42600.000000,0,0.482261,44351.349092,40629.650427,1,2
1156,42095.100000,0,0.482189,44351.349092,40629.650427,1,2
1157,42174.300000,1,0.482406,44351.349092,40629.650427,1,2
1158,42310.000000,1,0.561829,44351.349092,40629.650427,1,2


In [26]:
if not firebase_admin._apps:
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred)
    print("Firebase initialized")
db = firestore.client()

Firebase initialized


In [70]:
# getting prediction for current market data
prediction = model.predict(
    df[["mfi", "close_binary", "supertrend", "rsi_signals"]].tail(89)
)[0]
position_to_take = "hold"
if prediction == Positions.LONG:
    position_to_take = "BUY"
elif prediction == Positions.SHORT:
    position_to_take = "SELL"
print("position_to_take:", position_to_take)

position_to_take: BUY


In [71]:
# Checking if it's valid to trade
btc_last_price = getTickerDataForLastPrice()['open'].iloc[-1]
swing_low = df["sl_nums"].iloc[-1]
swing_high = df["sh_nums"].iloc[-1]
print('current_price:', btc_last_price)
print('swing_low:', swing_low)
print('swing_high:', swing_high)
print()
is_valid_to_trade = False
if position_to_take == "BUY":
    if btc_last_price > swing_low:
        is_valid_to_trade = True
if position_to_take == "SELL":
    if btc_last_price < swing_high:
        is_valid_to_trade = True
        
print("is_valid_to_trade:", is_valid_to_trade)

current_price: 46600.0
swing_low: 38580.52624069
swing_high: 43854.4

is_valid_to_trade: True


In [76]:
HOST_URL = "https://testnet.binancefuture.com"
END_POINT = "/fapi/v1/order"

# getting info from database
docs = (
    db.collection("agents")
    .where(filter=FieldFilter("position", "==", "hold"))
    .stream()
)
agents = []

for doc in docs:
    document = doc.to_dict()
    document["ref"] = doc.id
    agents.append(document)

if is_valid_to_trade:
    for i, agent in enumerate(agents):
        # Setting up request info
        api_key = agent["apiKey"]
        api_secret = agent["apiSecret"]
        amount_of_usd_to_use = agent["usdtToUse"] * (
            agent["percentagePerTrade"] * 0.01
        )
        converted_quantity = amount_of_usd_to_use / btc_last_price
        converted_quantity = round(float(converted_quantity), 3)
        data_query = f"symbol=BTCUSDT&side={position_to_take}&type=LIMIT&quantity={converted_quantity}&timeInForce=GTC&price={btc_last_price}&timestamp={str(time.time()).replace('.','')[0: 13]}"
        signed_hash_key = hmac.new(
            bytes(api_secret, "latin-1"),
            msg=bytes(data_query, "latin-1"),
            digestmod=hashlib.sha256,
        ).hexdigest()

        data_query = data_query + f"&signature={signed_hash_key}"
        url = HOST_URL + END_POINT + "?" + data_query

        # Sending request
        header = {"X-MBX-APIKEY": api_key, "signature": signed_hash_key}
        response = requests.post(url=url, headers=header)
        print(f"opening position for agent: {agent['associatedAccountName']} ------- {position_to_take}: " + response.text)
        trades_done.append(response.text)
        jsonified_response = response.json()
        
        if 'firstOrderId' not in agent.keys():
            agent['firstOrderId'] = jsonified_response['orderId']
            
        agent['currentOrderId'] = jsonified_response['orderId']

        target_price = 0
        stop_price = 0
        if position_to_take == "BUY":
            target_price = btc_last_price + (btc_last_price - swing_low) * 0.3
            stop_price = swing_low
            print(f"stop price for agent {agent['associatedAccountName']}: {stop_price}")
        elif position_to_take == "SELL":
            target_price = btc_last_price - (swing_high - btc_last_price) * 0.3
            stop_price = swing_high
            print(f"stop price for agent {agent['associatedAccountName']}: {stop_price}")

        # Setting up document for database
        reference_to_document = agent.pop("ref")
        agent["held_quantity"] = converted_quantity
        agent["price_of_asset"] = btc_last_price
        agent["position"] = position_to_take
        agent['target_price'] = target_price
        agent['stop_price'] = stop_price

        # Editing document in database
        db.collection("agents").document(reference_to_document).set(agent)

opening position for agent: bomba ------- BUY: {"orderId":3692832605,"symbol":"BTCUSDT","status":"NEW","clientOrderId":"ytFsSlI1Ghz21PwXGIq0Bb","price":"46600.00","avgPrice":"0.00","origQty":"0.016","executedQty":"0.000","cumQty":"0.000","cumQuote":"0.00000","timeInForce":"GTC","type":"LIMIT","reduceOnly":false,"closePosition":false,"side":"BUY","positionSide":"BOTH","stopPrice":"0.00","workingType":"CONTRACT_PRICE","priceProtect":false,"origType":"LIMIT","priceMatch":"NONE","selfTradePreventionMode":"NONE","goodTillDate":0,"updateTime":1707578023855}
stop price for agent bomba: 38580.52624069
opening position for agent: altatata ------- BUY: {"orderId":3692832616,"symbol":"BTCUSDT","status":"NEW","clientOrderId":"zbZyYqSeJ4KMZg9wRKCVRW","price":"46600.00","avgPrice":"0.00","origQty":"0.010","executedQty":"0.000","cumQty":"0.000","cumQuote":"0.00000","timeInForce":"GTC","type":"LIMIT","reduceOnly":false,"closePosition":false,"side":"BUY","positionSide":"BOTH","stopPrice":"0.00","workin

In [77]:
# getting info from database
docs = (
    db.collection("agents")
    .where(filter=FieldFilter("position", "!=", "hold"))
    .stream()
)
agents = []

for doc in docs:
    document = doc.to_dict()
    document["ref"] = doc.id
    agents.append(document)
    
print()

for i, agent in enumerate(agents):
    print(f"checking agent [{agent['associatedAccountName']}] for position closure")
    trade_done = False
    inverted_position = None
    trade_done_reason = None

    if agent['position'] == 'BUY':
        inverted_position = "SELL"
        if btc_last_price < agent['stop_price']:
            trade_done = True
            trade_done_reason = "Stop price reached"
        if btc_last_price > agent['target_price']:
            trade_done = True
            trade_done_reason = "Target price reached"
    if agent['position'] == "SELL":
        inverted_position = "BUY"
        if btc_last_price > agent['stop_price']:
            trade_done = True
            trade_done_reason = "Stop price reached"
        if btc_last_price < agent['target_price']:
            trade_done = True
            trade_done_reason = "Target price reached"

    if trade_done:
        print(f"Trade Done for {agent['associatedAccountName']}\nReason: {trade_done_reason}")
        # Setting up request info
        api_key = agent["apiKey"]
        api_secret = agent["apiSecret"]
        amount_of_usd_to_use = agent["usdtToUse"] * (
            agent["percentagePerTrade"] * 0.01
        )
        converted_quantity = agent["held_quantity"]
        data_query = f"symbol=BTCUSDT&side={inverted_position}&type=LIMIT&quantity={converted_quantity}&timeInForce=GTC&price={btc_last_price}&timestamp={str(time.time()).replace('.','')[0: 13]}"
        signed_hash_key = hmac.new(
            bytes(api_secret, "latin-1"),
            msg=bytes(data_query, "latin-1"),
            digestmod=hashlib.sha256,
        ).hexdigest()

        data_query = data_query + f"&signature={signed_hash_key}"
        url = HOST_URL + END_POINT + "?" + data_query

        # Sending request
        header = {"X-MBX-APIKEY": api_key, "signature": signed_hash_key}
        response = requests.post(url=url, headers=header)
        print(f"closing position for agent: {agent['associatedAccountName']} ------- {inverted_position}: " + response.text)
        trades_done.append(response.text)

        # Setting up document for database
        reference_to_document = agent.pop("ref")
        agent.pop("held_quantity")
        agent.pop("price_of_asset")
        agent.pop("target_price")
        agent.pop("stop_price")
        agent.pop("currentOrderId")
        if agent['position'] != 'inactive':
            agent["position"] = "hold"

        # Editing document in database
        db.collection("agents").document(reference_to_document).set(agent)     
    else:
        print(f"{agent['associatedAccountName']} is eligible for a new stop price")
        reference_to_document = agent.pop("ref")
        if agent['position'] == 'BUY':
            if swing_low > agent['stop_price']:
                agent['stop_price'] = swing_low
                db.collection("agents").document(reference_to_document).set(agent)
        elif agent['position'] == 'SELL':
            if swing_high < agent['stop_price']:
                agent['stop_price'] = swing_high
                db.collection("agents").document(reference_to_document).set(agent)


checking agent [bomba] for position closure
Trade Done for bomba
Reason: None
closing position for agent: bomba ------- SELL: {"orderId":3692833014,"symbol":"BTCUSDT","status":"NEW","clientOrderId":"7FP06GRv8MZyBaPudY3E5B","price":"46600.00","avgPrice":"0.00","origQty":"0.016","executedQty":"0.000","cumQty":"0.000","cumQuote":"0.00000","timeInForce":"GTC","type":"LIMIT","reduceOnly":false,"closePosition":false,"side":"SELL","positionSide":"BOTH","stopPrice":"0.00","workingType":"CONTRACT_PRICE","priceProtect":false,"origType":"LIMIT","priceMatch":"NONE","selfTradePreventionMode":"NONE","goodTillDate":0,"updateTime":1707578058527}
checking agent [altatata] for position closure
Trade Done for altatata
Reason: None
closing position for agent: altatata ------- SELL: {"orderId":3692833019,"symbol":"BTCUSDT","status":"NEW","clientOrderId":"B4udVGfIPos47Xp8166Bke","price":"46600.00","avgPrice":"0.00","origQty":"0.010","executedQty":"0.000","cumQty":"0.000","cumQuote":"0.00000","timeInForce":